<a href="https://colab.research.google.com/github/saksham0309/P2_WebNLG2020/blob/main/WebNLG_G2T_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Installing the required packages

In [ ]:
!pip install transformers
!pip install sentencepiece

     |████████████████████████████████| 3.4 MB 4.2 MB/s 
     |████████████████████████████████| 596 kB 64.7 MB/s 
     |████████████████████████████████| 895 kB 61.1 MB/s 
     |████████████████████████████████| 67 kB 2.8 MB/s 
     |████████████████████████████████| 3.3 MB 59.6 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
     |████████████████████████████████| 1.2 MB 4.3 MB/s 


## Importing required libraries

In [ ]:
import pandas as pd
import os
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration
from transformers.optimization import  Adafactor 
import time
import warnings
warnings.filterwarnings('ignore')

## Preprocess the data

In [ ]:
import urllib.request
import zipfile
url = 'https://gitlab.com/shimorina/webnlg-dataset/-/archive/master/webnlg-dataset-master.zip?path=release_v3.0/en/train'
urllib.request.urlretrieve(url, 'web.zip')
with zipfile.ZipFile('web.zip', 'r') as zip_ref:
    zip_ref.extractall('web')
import glob
import os
import re
import xml.etree.ElementTree as ET
import pandas as pd
files = glob.glob("/content/web/webnlg-dataset-master-release_v3.0-en-train/release_v3.0/en/train/**/*.xml", recursive=True)
triple_re=re.compile('(\d)triples')
data_dct={}
for file in files:
    tree = ET.parse(file)
    root = tree.getroot()
    triples_num=int(triple_re.findall(file)[0])
    for sub_root in root:
        for ss_root in sub_root:
            strutured_master=[]
            unstructured=[]
            for entry in ss_root:
                unstructured.append(entry.text)
                strutured=[triple.text for triple in entry]
                strutured_master.extend(strutured)
            unstructured=[i for i in unstructured if i.replace('\n','').strip()!='' ]
            strutured_master=strutured_master[-triples_num:]
            strutured_master_str=(' && ').join(strutured_master)
            data_dct[strutured_master_str]=unstructured
mdata_dct={"prefix":[], "input_text":[], "target_text":[]}
for st,unst in data_dct.items():
    for i in unst:
        mdata_dct['prefix'].append('webNLG')
        mdata_dct['input_text'].append(st)
        mdata_dct['target_text'].append(i)


df=pd.DataFrame(mdata_dct)
df.to_csv('webNLG2020_train.csv')

In [ ]:
train_df=pd.read_csv('webNLG2020_train.csv', index_col=[0])

Trimming off a few data points and so that a batch would not leave any remainder, hence some lines of codes can be avoided (Okay, this might be a hackish way of doing it )

In [ ]:
train_df=train_df.iloc[  :35000,:]

In [ ]:
train_df=train_df.sample(frac = 1)

In [ ]:
batch_size=8
num_of_batches=len(train_df)/batch_size
num_of_epochs=4

In [ ]:
num_of_batches=int(num_of_batches)

Checking for the GPU availability

In [ ]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0") 
    print("Running on the GPU")
else:
    dev = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


## Loading the pretrained model and tokenizer

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base', return_dict=True)
#moving the model to device(GPU/CPU)
model.to(dev)

Downloading:   0%|          | 0.00/773k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.32M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.17k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/850M [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

## Initializing the Adafactor optimizer with parameter values suggested for t5

In [ ]:

optimizer = Adafactor(
    model.parameters(),
    lr=1e-3,
    eps=(1e-30, 1e-3),
    clip_threshold=1.0,
    decay_rate=-0.8,
    beta1=None,
    weight_decay=0.0,
    relative_step=False,
    scale_parameter=False,
    warmup_init=False
)

  






In [ ]:

from IPython.display import HTML, display

def progress(loss,value, max=100):
    return HTML(""" Batch loss :{loss}
        <progress
            value='{value}'
            max='{max}',
            style='width: 100%'
        >
            {value}
        </progress>
    """.format(loss=loss,value=value, max=max))

In [ ]:
num_of_epochs=1

## Training the model

In [ ]:
# #Sets the module in training mode
# model.train()

# loss_per_10_steps=[]
# for epoch in range(1,num_of_epochs+1):
#   print('Running epoch: {}'.format(epoch))
  
#   running_loss=0

#   out = display(progress(1, num_of_batches+1), display_id=True)
#   for i in range(num_of_batches):
#     inputbatch=[]
#     labelbatch=[]
#     new_df=train_df[i*batch_size:i*batch_size+batch_size]
#     for indx,row in new_df.iterrows():
#       input = 'WebNLG: '+row['input_text']+'</s>' 
#       labels = row['target_text']+'</s>'   
#       inputbatch.append(input)
#       labelbatch.append(labels)
#     inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=400,return_tensors='pt')["input_ids"]
#     labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=400,return_tensors="pt") ["input_ids"]
#     inputbatch=inputbatch.to(dev)
#     labelbatch=labelbatch.to(dev)

#     # clear out the gradients of all Variables 
#     optimizer.zero_grad()

#     # Forward propogation
#     outputs = model(input_ids=inputbatch, labels=labelbatch)
#     loss = outputs.loss
#     loss_num=loss.item()
#     logits = outputs.logits
#     running_loss+=loss_num
#     if i%10 ==0:      
#       loss_per_10_steps.append(loss_num)
#     out.update(progress(loss_num,i, num_of_batches+1))

#     # calculating the gradients
#     loss.backward()

#     #updating the params
#     optimizer.step()
    
#   running_loss=running_loss/int(num_of_batches)
#   print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))
  

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:
#!ls /content/drive/MyDrive

In [ ]:
# model_save_name = 'classifier.pt'
# path = F"/content/drive/MyDrive/WebNLG/{model_save_name}" 
# torch.save(model.state_dict(), path)

In [ ]:
model_save_name = 'classifier.pt'
path = F"/content/drive/MyDrive/WebNLG/{model_save_name}"
model.load_state_dict(torch.load(path))

<All keys matched successfully>

## Plotting the loss over time

In [ ]:
# import matplotlib.pyplot as plt
   
# steps = [i*100 for i in range(len(loss_per_10_steps))]
  
# plt.plot(steps, loss_per_10_steps)
# plt.title('Loss')
# plt.xlabel('Steps')
# plt.ylabel('Loss')
# plt.show()

## Testing the model

In [ ]:
model.eval()
input_ids = tokenizer.encode("WebNLG: sidharth | hometown | Delhi && sidharth | play |  football </s>", return_tensors="pt")  # Batch size 1
                                    # Subject | relation | Subject && Location | Verb | Object
input_ids=input_ids.to(dev)
outputs = model.generate(input_ids)
tokenizer.decode(outputs[0])

## Results


In [ ]:
torch.save(model.state_dict(),'/content/drive/MyDrive/WebNLG/pytoch_model.bin')

In [ ]:
!wget https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json


--2022-01-18 13:28:34--  https://s3.amazonaws.com/models.huggingface.co/bert/t5-base-config.json
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.132.192
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.132.192|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1199 (1.2K) [application/json]
Saving to: ‘t5-base-config.json’

t5-base-config.json 100%[===================>]   1.17K  --.-KB/s    in 0s      

2022-01-18 13:28:35 (68.5 MB/s) - ‘t5-base-config.json’ saved [1199/1199]



In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model =T5ForConditionalGeneration.from_pretrained('/content/drive/MyDrive/WebNLG/pytoch_model.bin', 
                                                return_dict=True, config='t5-base-config.json')
def generate(text):
    model.eval()
    input_ids = tokenizer.encode("WebNLG:{} </s>".format(text), 
                               return_tensors="pt")  
    outputs = model.generate(input_ids)
    return tokenizer.decode(outputs[0])

In [ ]:
generate('Russia | leader | Putin')

'<pad> Putin is the leader of Russia.</s>'

In [ ]:
generate('Russia | leader | Putin')

'<pad> The Mason School of Business are the current tenants of Alan B Miller Hall, Williamsburg.</s>'

In [ ]:
generate('Sidharth | profession | Doctor && Sidharth | home_town | Bombay')

'<pad> Sidharth is a doctoral student who lives in Bombay.</s>'

In [ ]:
generate('Nie_Haisheng | birthDate | 1964-10-13 &8 Nie_Haisheng | occupation | Fighter_pilot')

'<pad> Born on October 13th, 1964, Nieheng was a fighter pilot.</s>'

In [ ]:
generate('Bananaman | creator | Steve_Bright && Bananaman | broadcastedBy | BBC')

'<pad> Bananaman, created by Steve Bright, was broadcast by the BBC.</s>'

In [ ]:
generate('Bananaman | lastaired | "1986-04-15" && Bananaman | creator | Steve_Bright')

'<pad> Bananaman was created by Steve Bright and last aired on 15th April 1986'

In [ ]:
generate('Alan_B._Miller_Hall | currentTenants | Mason_School_of_Business && Alan_B._Miller_Hall | location | Williamsburg')

'<pad> The Mason School of Business are the current tenants of Alan B Miller Hall, Williamsburg.</s>'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
